## **1. Research Questions introduction**



	“Quais são os fatores (género, número de ratings, e popularidade) que mais influenciam a média de avaliação dos filmes no MovieLens?”

Isto permite:
	•	Explorar a estrutura e a distribuição dos dados (EDA);
	•	Criar métricas derivadas (popularidade, número de votos, média ponderada);
	•	Usar modelos simples (regressão linear / árvore de decisão) para quantificar o impacto de cada variável;
	•	E escalar a análise — localmente em DuckDB, e facilmente replicável em cloud (Athena/BigQuery).

## **2. Pipeline**

### **2.1. Data Preparation**

1.	Carregar dados (DuckDB ou pandas).
	2.	Calcular:

	•	nº total de utilizadores, filmes e avaliações
	•	distribuição de rating (média, mediana, desvio padrão)
	•	nº de filmes por género
	•	nº médio de ratings por filme e por utilizador
	3.	Visualizar:
	
	•	histograma das classificações
	•	top 10 géneros mais avaliados
	•	relação entre nº de ratings e média por filme


### **2.2. Exploratory data analysis**

### **2.3. Feature Engineering**

Criar novas variáveis:
	•	n_ratings → nº de avaliações por filme
	•	avg_rating → média de rating por filme
	•	genre_count → nº de géneros atribuídos
	•	popularity_score = log(1 + nº de ratings) × média (para normalizar popularidade)

## **3. Models**

Objetivo: medir o peso de cada fator sobre a média de avaliação.
Modelos possíveis:
	•	Regressão Linear (OLS) → prever avg_rating com base em n_ratings, genre_count e dummies dos géneros principais.
	•	Árvore de decisão / Random Forest → avaliar a importância relativa das variáveis.
Métricas: R², RMSE, importância de features.


## **4. Visualization and Anaysis**

Gráficos:
	•	Scatter n_ratings vs avg_rating (com tendência)
	•	Boxplots de avg_rating por género
	•	Importância das features no modelo
	•	Insights:
	•	Géneros com maiores médias
	•	Géneros mais populares
	•	Relação entre popularidade e qualidade percebida


## **5. Escalability and big_data**

Executar as queries principais em DuckDB e exportar para Parquet.
	•	Demonstrar (mesmo que parcialmente) a execução de consultas analíticas escaláveis:

SELECT genre, COUNT(*) AS n_filmes, AVG(rating) AS avg_rating
FROM ratings
JOIN movies USING(movieId)
GROUP BY genre;

	•	Mostrar que a mesma lógica pode ser usada em Athena / BigQuery com datasets maiores (ex: MovieLens 25M).

## **5. Conclusions**

	•	Identificar que fatores explicam melhor as boas avaliações.
	•	Propor extensão:
	•	Modelo preditivo por utilizador (colaborativo)
	•	Integração com tags para enriquecer o conteúdo
